보고서, 뉴스 -> 노인에 맞게 쉽게 - LLM(HyperClovaX)

In [48]:
import json

input_file = 'sample.jsonl'
output_file = 'modified_sample.jsonl'

# 읽기 모드로 파일 열기
with open(input_file, 'r', encoding='utf-8') as f:
    data = f.readlines()

# 결과를 저장할 리스트
processed_data = []

# 각 줄에 대해 처리
for line in data:
    json_data = json.loads(line.strip())
    
    # 필요한 정보 추출
    article_text = ' '.join(json_data['article_original'])
    abstractive_text = json_data['abstractive']
    
    # 결과 포맷에 맞게 변환
    processed_entry = {
        "text": article_text,
        "completion": abstractive_text
    }
    
    # 리스트에 추가
    processed_data.append(processed_entry)

# 결과 파일 쓰기 400개만
cnt = 0
with open(output_file, 'w', encoding='utf-8') as f:
    for entry in processed_data:
        cnt += 1
        json.dump(entry, f, ensure_ascii=False)
        f.write('\n')
        if cnt == 20:
            break



In [47]:
len(processed_data)

3012

In [1]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import pandas as pd
import re

# 각 크롤링 결과 저장하기 위한 리스트 선언 
title_text = []
link_text = []
source_text = []
date_text = []
contents_text = []
result = {}

# 엑셀로 저장하기 위한 변수
RESULT_PATH = 'C:/Users/cksdu/OneDrive/문서/바탕 화면/FIND-A/FIT/미래에셋/'  # 결과 저장할 경로
now = datetime.now()  # 파일이름 현 시간으로 저장하기

# 날짜 정제화 함수
def date_cleansing(test):
    try:
        # 지난 뉴스
        pattern = '\d+.(\d+).(\d+).'  # 정규표현식 
        r = re.compile(pattern)
        match = r.search(test).group(0)  # 2018.11.05.
        date_text.append(match)
    except AttributeError:
        # 최근 뉴스
        pattern = '\w* (\d\w*)'  # 정규표현식 
        r = re.compile(pattern)
        match = r.search(test).group(1)
        date_text.append(match)

# 내용 정제화 함수 
def contents_cleansing(contents):
    first_cleansing_contents = re.sub('<dl>.*?</a> </div> </dd> <dd>', '', str(contents)).strip()  # 앞에 필요없는 부분 제거
    second_cleansing_contents = re.sub('<ul class="relation_lst">.*?</dd>', '', first_cleansing_contents).strip()  # 뒤에 필요없는 부분 제거 (새끼 기사)
    third_cleansing_contents = re.sub('<.+?>', '', second_cleansing_contents).strip()
    return third_cleansing_contents

# 개별 뉴스 링크에 접속하여 본문을 가져오는 함수
def get_article_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    content = soup.select_one('#articleBodyContents')  # 기사 본문에 해당하는 CSS 선택자
    if content:
        return contents_cleansing(content)
    return "본문을 가져올 수 없습니다."

def crawler(maxpage, query, sort, s_date, e_date):
    s_from = s_date.replace(".", "")
    e_to = e_date.replace(".", "")
    page = 1  
    maxpage_t = (int(maxpage)-1) * 10 + 1  # 11= 2페이지 21=3페이지 31=4페이지 ...

    while page <= maxpage_t:
        url = f"https://search.naver.com/search.naver?where=news&query={query}&sort={sort}&ds={s_date}&de={e_date}&nso=so%3Ar%2Cp%3Afrom{s_from}to{e_to}%2Ca%3A&start={str(page)}"
        
        response = requests.get(url)
        html = response.text
 
        soup = BeautifulSoup(html, 'html.parser')
 
        # <a> 태그에서 제목과 링크 주소 추출
        atags = soup.select('.news_tit')
        for atag in atags:
            title_text.append(atag.text)  # 제목
            link_text.append(atag['href'])  # 링크 주소

        # 신문사 추출
        source_lists = soup.select('.info_group > .press')
        for source_list in source_lists:
            source_text.append(source_list.text)  # 신문사
        
        # 날짜 추출
        date_lists = soup.select('.info_group > span.info')
        for date_list in date_lists:
            if date_list.text.find("면") == -1:
                date_text.append(date_list.text)

        # 각 뉴스 링크에 접속하여 본문 추출
        for link in link_text:
            article_content = get_article_content(link)
            contents_text.append(article_content)
        
        result = {
            "date": date_text,
            "title": title_text,
            "source": source_text,
            "contents": contents_text,
            "link": link_text
        }  
        print(page)
        
        df = pd.DataFrame(result)
        page += 10
    
    outputFileName = f'{now.year}-{now.month}-{now.day} {now.hour}시 {now.minute}분 {now.second}초 merging.xlsx'
    df.to_excel(RESULT_PATH + outputFileName, sheet_name='sheet1')


<>:24: SyntaxWarning: invalid escape sequence '\d'
<>:30: SyntaxWarning: invalid escape sequence '\w'
<>:24: SyntaxWarning: invalid escape sequence '\d'
<>:30: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4564\2030849212.py:24: SyntaxWarning: invalid escape sequence '\d'
  pattern = '\d+.(\d+).(\d+).'  # 정규표현식
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4564\2030849212.py:30: SyntaxWarning: invalid escape sequence '\w'
  pattern = '\w* (\d\w*)'  # 정규표현식


In [2]:

info_main = input("="*50+"\n"+"입력 형식에 맞게 입력해주세요."+"\n"+" 시작하시려면 Enter를 눌러주세요."+"\n"+"="*50)

# maxpage = input("최대 크롤링할 페이지 수 입력하시오: ")  
# query = input("검색어 입력: ")  
# sort = input("뉴스 검색 방식 입력(관련도순=0  최신순=1  오래된순=2): ")    #관련도순=0  최신순=1  오래된순=2
# s_date = input("시작날짜 입력(2019.01.04):")  #2019.01.04
# e_date = input("끝날짜 입력(2019.01.05):")   #2019.01.05

maxpage = "5"
query = "채권"
sort = "1"
s_date = "2019.01.04"
e_date = "2019.01.05"

crawler(maxpage,query,sort,s_date,e_date) 

1
11


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4564\2030849212.py:24: SyntaxWarning: invalid escape sequence '\d'
  pattern = '\d+.(\d+).(\d+).'  # 정규표현식
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_4564\2030849212.py:30: SyntaxWarning: invalid escape sequence '\w'
  pattern = '\w* (\d\w*)'  # 정규표현식


ValueError: All arrays must be of the same length

In [5]:
import re

pattern = '\d+.(\d+).(\d+).'
re.compile(pattern)

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17192\260667313.py:3: SyntaxWarning: invalid escape sequence '\d'
  pattern = '\d+.(\d+).(\d+).'


re.compile(r'\d+.(\d+).(\d+).', re.UNICODE)

In [4]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys

# 페이지 url 형식에 맞게 바꾸어 주는 함수 만들기
  #입력된 수를 1, 11, 21, 31 ...만들어 주는 함수
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num+9*(num-1)

# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)

def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        print("생성url: ", url)
        return url
    else:
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        print("생성url: ", urls)
        return urls    

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(i,headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url


#####뉴스크롤링 시작#####

#검색어 입력
search = input("검색할 키워드를 입력해주세요:")
#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")   
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")   


# naver url 생성
url = makeUrl(search,page,page2)

#뉴스 크롤러 실행
news_titles = []
news_url =[]
news_contents =[]
news_dates = []
for i in url:
    url = articles_crawler(url)
    news_url.append(url)


#제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

    
#제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

#1차원 리스트로 만들기(내용 제외)
makeList(news_url_1,news_url)

#NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

# 뉴스 내용 크롤링

for i in tqdm(final_urls):
    #각 기사 html get하기
    news = requests.get(i,headers=headers)
    news_html = BeautifulSoup(news.text,"html.parser")

    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
    
    # 뉴스 본문 가져오기
    content = news_html.select("article#dic_area")
    if content == []:
        content = news_html.select("#articeBody")

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ",(page2+1-page)*10,'개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)

print('news_title: ',len(news_titles))
print('news_url: ',len(final_urls))
print('news_contents: ',len(news_contents))
print('news_dates: ',len(news_dates))

###데이터 프레임으로 만들기###
import pandas as pd

#데이터 프레임 만들기
news_df = pd.DataFrame({'date':news_dates,'title':news_titles,'link':final_urls,'content':news_contents})

#중복 행 지우기
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)
print("중복 제거 후 행 개수: ",len(news_df))

#데이터 프레임 저장
now = datetime.datetime.now() 
news_df.to_csv('{}_{}.csv'.format(search,now.strftime('%Y%m%d_%H시%M분%S초')),encoding='utf-8-sig',index=False)


크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지:  1 페이지
생성url:  https://search.naver.com/search.naver?where=news&sm=tab_pge&query=채권&start=1


MissingSchema: Invalid URL 'h': No scheme supplied. Perhaps you meant https://h?